# VLM

# RAG

- Храним данные в Chroma, collection_name="dota_knowledge_1.3"
- На данный момент там небольшая справка про каждого из героев доты с wikifandom, данные повторяются

## Собираем данные

### Качаем библиотеки

In [ ]:
!pip install langchain_chroma langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 123.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.1/467.1 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 91.8 MB/s eta 0:00:

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import nltk
import json

from pathlib import Path
from pprint import pprint

import subprocess
import os

from langchain_community.document_loaders import JSONLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import BSHTMLLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader

from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_text_splitters import CharacterTextSplitter
import chromadb
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import BSHTMLLoader
from langchain.text_splitter import CharacterTextSplitter

from langchain.text_splitter import RecursiveCharacterTextSplitter

### Дообучение энкодера под домен -- coming soon

In [ ]:
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer, losses
from sentence_transformers.readers import InputExample
from torch.utils.data import DataLoader


train_examples = #?

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

model = SentenceTransformer('all-MiniLM-L6-v2')

train_loss = losses.MultipleNegativesRankingLoss(model)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=3,
    warmup_steps=100,
    output_path='./dota2-encoder'
)

### Парсим данные

In [ ]:
os.makedirs('dota_wget', exist_ok=True)

In [ ]:
dota_heroes = dota_heroes = [
    # Strength Heroes
    "Abaddon",
    "Alchemist",
    "Axe",
    "Bristleback",
    "Centaur_Warrunner",
    "Chaos_Knight",
    "Clockwerk",
    "Dawnbreaker",
    "Doom",
    "Dragon_Knight",
    "Earth_Spirit",
    "Earthshaker",
    "Elder_Titan",
    "Huskar",
    "Io",
    "Kunkka",
    "Legion_Commander",
    "Lifestealer",
    "Lycan",
    "Magnus",
    "Marci",
    "Mars",
    "Night_Stalker",
    "Ogre_Magi",
    "Omniknight",
    "Phoenix",
    "Pudge",
    "Sand_King",
    "Slardar",
    "Snapfire",
    "Sven",
    "Tidehunter",
    "Timbersaw",
    "Tiny",
    "Treant_Protector",
    "Tusk",
    "Underlord",
    "Undying",
    "Wraith_King",

    # Agility Heroes
    "Anti-Mage",
    "Arc_Warden",
    "Bloodseeker",
    "Bounty_Hunter",
    "Clinkz",
    "Drow_Ranger",
    "Ember_Spirit",
    "Faceless_Void",
    "Gyrocopter",
    "Hoodwink",
    "Juggernaut",
    "Lone_Druid",
    "Luna",
    "Medusa",
    "Meepo",
    "Mirana",
    "Monkey_King",
    "Morphling",
    "Naga_Siren",
    "Nyx_Assassin",
    "Pangolier",
    "Phantom_Assassin",
    "Phantom_Lancer",
    "Razor",
    "Riki",
    "Shadow_Fiend",
    "Slark",
    "Sniper",
    "Spectre",
    "Templar_Assassin",
    "Terrorblade",
    "Troll_Warlord",
    "Ursa",
    "Vengeful_Spirit",
    "Venomancer",
    "Viper",
    "Weaver",

    # Intelligence Heroes
    "Ancient_Apparition",
    "Crystal_Maiden",
    "Dark_Seer",
    "Dark_Willow",
    "Death_Prophet",
    "Disruptor",
    "Enchantress",
    "Enigma",
    "Grimstroke",
    "Invoker",
    "Jakiro",
    "Keeper_of_the_Light",
    "Leshrac",
    "Lich",
    "Lina",
    "Lion",
    "Nature's_Prophet",
    "Necrophos",
    "Oracle",
    "Outworld_Destroyer",
    "Puck",
    "Pugna",
    "Queen_of_Pain",
    "Rubick",
    "Shadow_Demon",
    "Shadow_Shaman",
    "Silencer",
    "Skywrath_Mage",
    "Storm_Spirit",
    "Techies",
    "Tinker",
    "Visage",
    "Void_Spirit",
    "Warlock",
    "Windranger",
    "Winter_Wyvern",
    "Witch_Doctor",
    "Zeus",

    # Universal Heroes
    "Muerta"
]

In [ ]:
def download_with_wget(hero, item=''):
  commands = [
        'wget', f'https://dota2.fandom.com/wiki/{hero}/{item}',
        '-O', f'dota_wget/{hero}{item}.html',
        '--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    ]

  try:
    result = subprocess.run(commands, capture_output=True, text=True)
    if result.returncode != 0:
        print(f'https://dota2.fandom.com/wiki/{hero}/{item} has an error {result.returncode}')
  except:
    ...

In [ ]:
traits = ['Guide', 'Equipment', 'Talents']

In [ ]:
import time

for hero in dota_heroes:
  for item in traits:
    download_with_wget(hero, item)
    time.sleep(4.3)

In [2]:
def merge_text_files(file_paths, output_file):
    merged_content = ""

    for i, file_path in enumerate(file_paths):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                merged_content += content
                print(f"Добавлен: {file_path}")

        except Exception as e:
            print(f"Ошибка чтения {file_path}: {e}")

    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(merged_content)

    return output_file

for hero in dota_heroes:
    merge_text_files([f'/content/dota_wget/{hero}.html', f'/content/dota_wget/{hero}Guide.html',
                     f'/content/dota_wget/{hero}Equipment.html', f'/content/dota_wget/{hero}Talents.html'],
                     f'/content/dota_wget/{hero}Total.html')

NameError: name 'dota_heroes' is not defined

## Делаем RAG

### Кодируем их и грузим в Chroma

In [ ]:
class DotaVectorStore:
    def __init__(self, model_name="all-MiniLM-L6-v2"):
        self.client = chromadb.PersistentClient(path="./dota_db_1.3")
        self.embedding_model = SentenceTransformer(model_name)
        self.doc_counter = 0
        self.collection = self.client.get_or_create_collection(
            name="dota2_knowledge_1.3",
            metadata={"description": "Dota 2 game knowledge base"}
        )
    def _embed_documents(self, texts):
        return self.embedding_model.encode(texts).tolist()

    def _embed_query(self, text):
        return self.embedding_model.encode([text]).tolist()[0]

    def add_document(self, document_html, metadatas=None):
        loader = BSHTMLLoader(document_html)
        data = loader.load()
        text_splitter = CharacterTextSplitter(chunk_size=700, chunk_overlap=100)
        docs = text_splitter.split_documents(data)
        texts = [doc.page_content for doc in docs]
        embeddings = self._embed_documents(texts)
        start_id = self.doc_counter
        ids = [f"doc_{start_id + i}" for i in range(len(texts))]
        self.doc_counter += len(texts)
        self.collection.add(
            documents=texts,
            embeddings=embeddings,
            metadatas=metadatas,
            ids=ids
        )

        print(f"Добавлено {len(texts)} документов")

    def search(self, query, n_results=5):
        query_embedding = self._embed_query(query)
        results = self.collection.query(
            query_embeddings=[query_embedding],
            n_results=n_results
        )

        return results

In [ ]:
rag_store = DotaVectorStore()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
for hero in dota_heroes:
    rag_store.add_document(f'/content/dota_wget/{hero}Total.html')

NameError: name 'dota_heroes' is not defined

### Проверим как работает RAG

In [ ]:
def list_all_documents(collection_name="dota2_knowledge_1.3", persist_directory="./dota_db_1.3"):
    import chromadb

    client = chromadb.PersistentClient(path=persist_directory)
    collection = client.get_collection(collection_name)

    results = collection.get()

    print(f"📚 Коллекция: {collection_name}")
    print(f"📄 Всего документов: {len(results['ids'])}")
    print("=" * 60)

    for i, (doc_id, document, metadata) in enumerate(zip(
        results['ids'],
        results['documents'],
        results['metadatas']
    )):
        print(f"\n{i+1}. 🆔 ID: {doc_id}")
        print(f"   🏷️  Метаданные: {metadata}")
        print(f"   📝 Превью: {document[:100]}...")
        print("-" * 50)

    return results

# documents_info = list_all_documents()

In [ ]:
rag_store.search('Who is Alchemist', 3)

{'ids': [['doc_29', 'doc_21', 'doc_25']],
 'embeddings': None,
 'documents': [["Manta Style is excellent for sieging buildings and taking waves, particularly in conjunction with  Radiance. It also provides Alchemist with a much needed disjoint mechanism.\n Mjollnir increases Alchemist's attack speed greatly, and also gives a useful Static Charge to protect Alchemist or his teammates. Its lightning attack will also trigger often under Chemical Rage.\n Battle Fury allows Alchemist to farm much quicker.\n Blade Mail forces enemies to think twice about fighting back against Alchemist.\n Abyssal Blade goes well with Alchemist's high attack speed, and helps greatly in securing an enemy. It also gives bonus attack damage and greater survivability providing bonus HP and blocking damage from incoming attacks.\n Heaven's Halberd gives some strength, evasion, status resistance, and also gives the ability to disarm a threatening enemy.\n Heart of Tarrasque gives Alchemist a massive amount of healt

In [ ]:
from vlm_functions import get_hero_name_from_image, recognize_heroes, restart_training

hero_name = get_hero_name_from_image("./test_pic/sniper.png")
print(f"Найденный герой: {hero_name}")

if hero_name != "Unknown":
    rag_results = rag_store.search(f"Who is {hero_name}? What are their abilities?", n_results=3)

    for doc in rag_results['documents'][0]:
        print(doc)

Найденный герой: Sniper
Tips & Tactics[]
General[]
     Stutter stepping between attack animations allows Sniper to land more hits on a moving target.  
Sniper can be played either in the safe lane with a support, as a solo mid, or as a soft support.
Sniper's purpose in fights is mostly to right click and attack enemies without being disabled or killed first.
Due to his extreme vulnerability, positioning is the most important part of playing Sniper.
Because of this, Sniper is easy to play, but hard to master.
Map awareness is very important for Sniper players, who should always mentally keep track of enemy positions in order to avoid being ganked.
Sniper players must know how to stutter step, moving small distances between attack animations in order to be mobile and land as many hits as possible. This is more important in the early game when Sniper's attack speed is slower.
Sniper must avoid being initiated on because an initiation and followup combination usually kills him before he c